In [1]:
### lHC-MR simulation script 
## Generating n data simulations
## Input: set up the parameters corresponding to the scenario, bolean for more complex scenarios,
# boolean indicating detailed saving or not
## Output: betXY, thS (parameters), sig1 + pi1 (local LD structure), nX, nY, m0 (number of SNPs in local region), 
# piU (constant), if desired: hrX + hrU + hrY

library(data.table)
library(rhdf5)
library(lhcMR)
library(dplyr)
library(PearsonDS)
library(jsonlite)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [394]:


#### start to change
## set working directory

## scenario number
dS = 1;
## data generations for this scenario
nsimi = 1
## boolean set up to indicate detailed save (with hrX/U/Y), kurtosis (non-normal scenario), or presence of two confounders (twoU)
detailedSave = FALSE
kurtosis = FALSE #if true change kurtosis level in first if clause below
twoUs = FALSE #if true change two U properties in firs if clause below
#### end to change

## read in pi1 and sig1 + R matrix
#Rmat = fread("LD_vsGM2.txt")
#c1 = fread("pi1-sigma1_vsGM2.txt")

# 1. Create a list of 1000 1's called Rmat
Rmat <- rep(1, 1000)

# 2. Create the pi1 array
pi1 <- rep(1, 1000)
#set.seed(123)  # Optional: for reproducibility
#pi1 <- abs(rnorm(1000, mean=0.5, sd=0.2))

# 3. Create the sig1 array
sig1 <- rep(0, 1000)
#sig1 <- abs(rnorm(1000, mean=0.3, sd=0.1))

sim_num = length(sig1);  # number of genotyped SNPs
m0 = 1;  # n_local true sequence variants in each SNP region - used for pi1/sig1
#rm(c1)#to save space

#### start to change

num_X1 = 1
num_X2 = 3
num_X3 = 6

## create bX bY from these parameters
piX = 10/1000;
piU = 100/1000;
piY = 100/1000;
h2X = .25;
h2U = .3;
h2Y = .2;
q1 <- abs(rnorm(n=num_X1, mean=0.0, sd=0.0))
q2 <- abs(rnorm(n=num_X2, mean=0.0, sd=0.0))
q3 <- abs(rnorm(n=num_X3, mean=0.0, sd=0.0))
qY = 0.0;
alp1 <- rnorm(n=num_X1, mean=0.3, sd=0.0)
alp2 <- rnorm(n=num_X2, mean=0.3, sd=0.0)
M = 1000;
sigX = sqrt(h2X/(M*piX));
sigU = sqrt(h2U/(M*piU));
sigY = sqrt(h2Y/(M*piY));
tX = q1*sqrt(h2U);
tY = qY*sqrt(h2U);
nX = 5e5;
nY = 5e5;
iX = 1;
iY = 1;
iXY = 0;



vx1 = 1
vx2 = 1
vx3 = 1

sdX1 = 0.5
sdX2 = 0.5
sdX3 = 0.5

#### end to change
system("mkdir data-mat")
output <- c()

In [395]:
print(alp2)

[1] 0.3 0.3 0.3


In [396]:
sim_hr <- function(sim_num, m0, pi, sig) {
  # Initialize hX as zeros
  hX <- matrix(0, nrow = sim_num, ncol = m0)
  
  # Randomly select pi unique indices from 1 to sim_num
  selected_indices <- sample(1:sim_num, pi, replace = FALSE)
  
  # Set the values at those indices to be 1
  hX[selected_indices, ] <- 1
  
  # Finding the indices where hX equals 1
  seli <- which(hX == 1, arr.ind = TRUE)
  
  # Generating normal random values
  tmp <- rnorm(nrow(seli), mean = 0, sd = sig)
  
  # Replacing the values at the specified indices with tmp values
  hX[seli] <- tmp
  
  # Calculating the sum across rows after element-wise multiplication
  # Assuming Rmat is globally available or passed as an argument to the function
  hrX <- rowSums(Rmat * hX)
  
  return(hrX)
}

In [397]:
# Generating lists of hr values for X1, X2, and X3 using the sim_hr function
hrX1_list <- lapply(1:num_X1, function(idx) sim_hr(sim_num, m0, piX*sim_num, sigX))
hrX2_list <- lapply(1:num_X2, function(idx) sim_hr(sim_num, m0, piX*sim_num, sigX))
hrX3_list <- lapply(1:num_X3, function(idx) sim_hr(sim_num, m0, piX*sim_num, sigX))

# Simulating hr values for U and Y
hrU <- sim_hr(sim_num, m0, piU*sim_num, sigU)
hrY <- sim_hr(sim_num, m0, piY*sim_num, sigY)

In [398]:
print(hrU)

   [1]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   [6]  0.000000e+00 -3.581253e-02  0.000000e+00  0.000000e+00  0.000000e+00
  [11]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  [16]  0.000000e+00  0.000000e+00  0.000000e+00 -9.543000e-03  0.000000e+00
  [21]  0.000000e+00 -1.842404e-02  0.000000e+00  0.000000e+00  0.000000e+00
  [26]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  [31]  0.000000e+00  0.000000e+00  0.000000e+00 -2.228164e-02  0.000000e+00
  [36]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00 -4.547749e-02
  [41]  0.000000e+00  6.287239e-02  0.000000e+00  0.000000e+00  0.000000e+00
  [46]  0.000000e+00  0.000000e+00  1.317817e-01  8.111248e-03  0.000000e+00
  [51]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  [56]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  [61]  0.000000e+00  5.236561e-02  0.000000e+00  0.000000e+00  0.000000e+00

In [399]:

# Calculating betX1 values.
betX1 <- lapply(1:length(hrX1_list), function(idx) {
  hrX1 <- hrX1_list[[idx]]
  hrX1 + q1[idx]*hrU + rnorm(length(hrX1), mean = 0, sd = sdX1)
})

# Calculating betX3 values.
betX3 <- lapply(1:length(hrX3_list), function(idx) {
  hrX3 <- hrX3_list[[idx]]
  hrX3 + q3[idx]*hrU + rnorm(length(hrX3), mean = 0, sd = sdX3)
})

# Calculating betY values.
betY <- (colSums(matrix(unlist(hrX1_list), nrow=length(hrX1_list[[1]]), ncol=length(hrX1_list)) * alp1) +
         (sum(alp1 * q1) + qY) * hrU + hrY) + rnorm(length(hrY), mean = 0, sd = sdX3)

# Calculating betX2 values.
betX2 <- lapply(1:length(hrX2_list), function(idx) {
  hrX2 <- hrX2_list[[idx]]
  (q2[idx] + alp2[idx] * qY + alp2[idx] * sum(alp1 * q1)) * hrU + 
    alp2[idx] * hrY + 
    alp2[idx] * colSums(matrix(unlist(hrX1_list), nrow=length(hrX1_list[[1]]), ncol=length(hrX1_list)) * alp1) + 
    hrX2 + 
    rnorm(length(hrX2), mean = 0, sd = sdX3)
})



In [400]:
print(betX2)

[[1]]
   [1]  0.3789917335  0.7680884504 -0.1846681921  0.6658854289 -0.2156108050
   [6]  0.1274594464 -0.1015868918 -1.0550773416  0.0728608436 -0.4201673600
  [11] -0.1414248305  0.3165297466 -0.6155260299 -0.8422236338  0.4618679390
  [16] -0.2837236156 -0.2273335547  1.0943790317 -0.7056064302 -0.3274822460
  [21] -0.4955168402 -0.5323131163  0.2036276693 -0.0881426368 -1.3374780233
  [26] -0.4572017266 -0.6630511657  0.4542672465 -0.3958627078  0.0309542026
  [31]  0.1488451687 -0.5169183316  0.5390063952 -0.6309601970  0.6588130319
  [36]  0.1451734857 -0.1782896232 -0.3877116621 -0.2401366642 -0.0005257311
  [41]  0.0621255986  0.0590279020  0.1871163539  0.3461793581  0.1523334518
  [46]  0.4623638236 -0.2579690800  1.2682211666  0.1189464313  0.4475175584
  [51] -1.0967936012 -0.6715663781 -0.2314243812  0.6061591912  0.5784717503
  [56] -0.2336846831  0.4485228757  0.3868012599  0.3374335931 -0.9181676760
  [61]  0.1010372402 -0.0047132544  0.1512485895  0.0940800826  0.8156

In [401]:

simi = 1
  message(simi)
  
  #betXY = cbind(betX,betY) # [betX,betY];
  
  #save parameters set up
  thS = c(piX,piY,sigX,sigY,tX,tY,alp1,alp2,iX,iY,iXY)


  save(betX1, betX2, betX3, betY, thS, sig1, pi1, nX, nY, m0, piU, file = paste0("./data-mat/exampleDataB",dS,"_sim",simi,".RData"))





1



In [2]:
########OPTIMISATION###########

In [37]:
dS = 1 #scenario number
nsimi = 1 #number of data generations from previous scripts
nSP = 10 #number of starting points for each generation
saveSP = TRUE #to save the starting points used for each data generation
paral_method = "lapply" #"lapply"
#### end to change

## single trait analysis to estimate pi, h2, i, followed by generating 50 SP for each data generation and optimisation
SP = 10; #number of starting points for single trait analysis to estimate pi, h2 and i
M = 100; #number of SNPs
nCores=1; #to set up for optimisation in lapply


In [38]:
simi=1 #Only one simulation, we're cheesing the for loop

In [39]:
  #load(paste0("./data-mat/exampleDataB",dS,"_sim",simi,".RData")) 
  #wd = rep(1, length(sig1)) #ones(length(sig1),1);
  #piX = thS[1];
  #piY = thS[2];
  #sigX = thS[3];
  #sigY = thS[4];
  #tX = thS[5];
  #tY = thS[6];
  #alp1 = thS[7];
  #alp2 = thS[8];
  #iX = thS[9];
  #iY = thS[10];
  #iXY = thS[11];
  #h2X = piX*M*sigX^2;
  #h2Y = piY*M*sigY^2;
  #herX = h2X+tX^2;
  #herY = h2Y+tY^2;


In [43]:
# Convert string representation to numeric vector
convert_string_to_numeric_vector <- function(s) {
  s <- gsub("\\[|\\]", "", s) # Remove [ and ]
  s_list <- unlist(strsplit(s, ","))
  return(as.numeric(s_list))
}

In [71]:
# Read the CSV file into a data frame
df <- read.csv('duckumeme.csv', header=TRUE)

# Create the thS variable as a list from the column named 'thS'
thS <- convert_string_to_numeric_vector(df$thS[[1]])

# Create the individual variables based on thS
piX <- thS[1]
piY <- thS[2]
sigX <- thS[3]
sigY <- thS[4]
tX <- thS[5]
tY <- thS[6]
alp1 <- thS[7]
alp2 <- thS[8]
iX <- thS[9]
iY <- thS[10]
iXY <- thS[11]

nX <- as.numeric(df$nX[[1]])
nY <- as.numeric(df$nY[[1]])
m0 <- as.numeric(df$m0[[1]])

wd = rep(1, M) #ones(length(sig1),1);

In [73]:
alp1

[1] 0.1

In [74]:
iXY=1

In [75]:
# Apply the conversion to beta_X1 column
betX1 <- lapply(df$betX1, convert_string_to_numeric_vector)
betX2 <- lapply(df$betX1, convert_string_to_numeric_vector)
betX3 <- lapply(df$betX1, convert_string_to_numeric_vector)
betY <- convert_string_to_numeric_vector(df$betY[[1]])



In [76]:
print(betX1)

[[1]]
  [1] -8.345479e-03  6.976634e-04  7.536529e-04 -7.488906e-04  4.409687e-03
  [6]  6.442677e-05 -3.596902e-01 -5.586241e-03  3.281783e-04  3.051350e-03
 [11] -7.948484e-03 -5.361403e-03  2.738100e-02  1.219303e-04 -4.747302e-03
 [16] -3.496061e-04  3.642755e-03 -1.817235e-03 -6.357011e-03 -2.780906e-04
 [21] -3.041643e-03 -3.787008e-03  7.766551e-03  1.558788e-03 -1.325635e-03
 [26] -2.824839e-03 -1.084907e-02 -3.042183e-03  3.346588e-03  8.477545e-02
 [31]  3.118291e-03  1.695335e-03 -1.606167e-03  1.064372e-01 -1.175674e-03
 [36]  4.025853e-03 -3.429656e-03  3.139576e-03  1.740684e-03  9.827008e-03
 [41] -1.158779e-02  2.967661e-03 -2.557334e-03 -4.739532e-03 -2.008980e-05
 [46]  4.996952e-03 -2.217161e-03 -3.230759e-04  5.303509e-03 -4.317409e-03
 [51] -8.812392e-02 -1.379820e-03 -9.760561e-03 -1.354368e-03 -6.066235e-03
 [56]  7.689389e-03  1.123847e-03  3.080593e-03 -3.525880e-03  2.049691e-03
 [61] -5.869668e-03 -1.866667e-01  8.167519e-04  2.328194e-03 -3.835999e-03
 [66] 

In [77]:
  ## SP calculation
  sp_piX = runif(SP,0.001,0.2)
  sp_h2X = runif(SP,0.15,0.5)
  sp_iX = runif(SP,0.5,1.5)


In [78]:
print(sp_piX)
print(sp_h2X)
print(sp_iX)

 [1] 0.07338965 0.05902738 0.06099845 0.01991257 0.01414131 0.16501265
 [7] 0.08547483 0.12961171 0.04290999 0.11092267
 [1] 0.3959331 0.2560438 0.3266440 0.2600885 0.4071973 0.2582012 0.4764730
 [8] 0.2934926 0.2227101 0.2961486
 [1] 0.5932808 1.3946423 1.4942358 0.8450262 0.7029825 1.0242617 1.0254718
 [8] 1.1062304 1.4510582 0.5605344


In [79]:
  # Get piX, iX (and total_h2) in a separate step - single trait analysis for X and Y
  para=cbind(sp_piX,sp_h2X,sp_iX)
  sp_mat=matrix(unlist(para), nrow=SP, ncol=3, byrow = FALSE)
  colnames(sp_mat)=colnames(para)
  par.df = data.frame(par=I(apply(sp_mat,1,as.list))) #generate a dataframe of lists for each row of parameters - input for rslurm/lapply
  

In [80]:
singleTrait_likelihood <- function(theta,betX,w8s,m0,nX,bn=2^7,bins=10,M=1e7){

  piX = abs(theta[1]);
  h2X = abs(theta[2]);
  iX = abs(theta[3]);

  sigX = sqrt(h2X/(piX*M))
  # Number of genotyped SNPs
  m = length(betX)

  Rp = iX/nX
  bX = array(betX, c(1,m)) # reshape(betXY(:,1),[1,1,m]);

  # Define grid for FFT
  minX = mean(bX)-(5*sd(bX));
  maxX = mean(bX)+(5*sd(bX));
  dX = (maxX-minX)/(bn-1);
  minX = minX-dX/2;
  maxX = maxX+dX/2;

  bXi = ceiling((bX-minX)/dX);
  bXi[bXi<1] = 1;
  bXi[bXi>bn] = bn;


  if(piX > 0.2 || piX < 1e-6 || h2X > 1 || h2X < 1e-6 || iX > 1.5 || iX < 0.5 ){
    logL = 1e10
  }else{
    # min_pi1 = min(pi1)-1e-10;
    # max_pi1 = max(pi1)+1e-10;
    # dp = (max_pi1-min_pi1)/bins;
    # pc = min_pi1 + (dp * matrix(seq(0.5,(bins-0.5),1),ncol=1))
    # pix = ceiling((pi1-min_pi1)/dp);
    # min_sig1 = min(sig1)-1e-10;
    # max_sig1 = max(sig1)+1e-10;
    # ds = (max_sig1-min_sig1)/bins;
    # sc = min_sig1 + (ds * matrix(seq(0.5,(bins-0.5),1),ncol=1))
    # six = ceiling((sig1-min_sig1)/ds);
    # cix = pix + bins*(six-1);
    # uni_cix = sort(unique(cix))
    # ucix = match(uni_cix, cix)
    # ixMap = match(cix, uni_cix)
    # Sig1 = sc[six[ucix]];
    # Pi1 = pc[pix[ucix]];
    # mm = length(Sig1);

    # Ax = aperm( array(rep((1/sigX) / Sig1, bn), c(mm,bn)), c(2,1))
    # Qx = aperm( array(rep(Pi1 * piX, bn), dim = c(mm, bn)),c(2,1))
    Ax = rep(sigX, bn)
    Qx = rep(piX, bn)


    j = 0:(bn-1)
    vi = 2*pi*(j-bn/2)/(maxX-minX-dX);

    # Rx = array(rep(vi, mm), c(bn,mm))
    Rx = array(c(vi), c(bn))

    # Lx = -m0 * ( 1 - 1 / sqrt(1 + Rx^2/Ax^2))*Qx
    Lx = -m0 * ( 1 - exp(-0.5*Ax^2*Rx^2))*Qx
    Le = -(1/2)*(Rp*Rx^2)

    # /!\ complex numbers here!
    mf_init = -2 * log(as.complex(-1)) * ( (minX+dX/2) / (maxX-minX-dX) )*j
    # mf = array(rep(mf_init, mm), dim=c(bn, mm))
    mf = array(c(mf_init), dim=c(bn))

    phi = exp(Lx+Le+mf);

    # In R, not possible to chose dimension for FFT, so we need a loop to do it for all rho bins
    # FFT=array(NA, dim=c(bn, mm))
    FFT=array(NA, dim=c(bn))
    # for(l in 1:mm){
    #   FFT[,l] = fft(phi[,l])
    # }
    FFT = fft(phi)

    FFTmod_init = (1/(maxX-minX-dX))*(as.complex(-1))^(bn*((minX+dX/2)/(maxX-minX-dX)) + j)
    # FFTmod = array(rep(FFTmod_init, mm), dim=c(bn,mm))
    FFTmod = array(c(FFTmod_init), dim=c(bn))

    FFT0 = Re(FFT*FFTmod)
    # pfE = FFT0[cbind(t(bXi), ixMap)];
    pfE = FFT0[t(bXi)];
    length(which(pfE<0))
    # If some, remove them & update weights to keep the correct set of SNPs
    my_w8s = w8s[pfE>0]
    pfE=pfE[pfE>0]

    # We use m * mean(...) to account for SNPs that may have been excluded before
    logL = -m * mean(log(pfE*my_w8s))
  }
  return(logL)
}


In [81]:
test_exp_function <- function(betX_value, n_val) {
  test.exp <- lapply(par.df[[1]], function(x) {
    theta = unlist(x) 
    test1 = optim(theta, singleTrait_likelihood,
                  betX = betX_value, w8s = wd, M = M,
                  m0 = m0, nX = n_val, bn = 2^7, bins = 10,
                  method = "Nelder-Mead",
                  control = list(maxit = 5e3))
    
    list("mLL" = test1$value, "par" = test1$par, "conv" = test1$convergence)
  })
  
  test.exp = as.data.frame(t(matrix(unlist(test.exp), nrow = length(unlist(test.exp[1])))))
  
  return(test.exp)
}

In [82]:
run_on_each_bet <- function(bet_list, n_val) {
    print(n_val)
  test.exp.list <- lapply(bet_list, function(bet_value) {
    test_exp_function(bet_value, n_val)
  })
  
  return(test.exp.list)
}

In [83]:
print(nY)

[1] 50000


In [84]:
test.exp.X1 = run_on_each_bet(betX1, nX)
test.exp.X2 = run_on_each_bet(betX2, nX)
test.exp.X3 = run_on_each_bet(betX3, nX)
test.exp.Y = test_exp_function(betY, nY)

[1] 50000
[1] 50000
[1] 50000


In [85]:
print(test.exp.Y)

          V1          V2          V3        V4 V5
1  -227.3358 0.175797941  0.46408874 0.5000003  0
2  -246.5448 0.199999918  0.22369126 1.4999999  0
3  -246.7942 0.199999986  0.17757376 1.4999999  0
4  -278.4361 0.199999981 -0.01336014 0.7695241  0
5  -271.3018 0.002792806  0.62717079 0.5988604  0
6  -246.7897 0.199999761  0.17082493 1.5000000  0
7  -246.7910 0.200000000  0.18103916 1.4999850  0
8  -273.3250 0.199999994 -0.01263501 1.1692350  0
9  -246.7945 0.199999973  0.17667208 1.4999998  0
10 -227.3571 0.163541482  0.35716166 0.5000026  0


In [86]:
# The function to process test results
process_test_results <- function(test_result) {
  colnames(test_result) <- c("mLL","piX", "h2X","iX","conv")
  
  # Get the row(s) with the minimum mLL for test.result
  res_min <- test_result[which(test_result$mLL == min(test_result$mLL)), ]
  
  # If there's more than one, only keep the first one
  if (nrow(res_min) > 1) {
    res_min <- res_min[1, ]
  }
  
  return(abs(res_min[2:4])) # piX, h2X, iX
}
# Modified function to process each data frame inside a list
process_test_results_list <- function(test_result_list) {
  processed_results <- lapply(test_result_list, process_test_results)
  return(processed_results)
}

In [87]:
# Process the test results
res_exp.X1.list <- process_test_results_list(test.exp.X1)
res_exp.X2.list <- process_test_results_list(test.exp.X2)
res_exp.X3.list <- process_test_results_list(test.exp.X3)

# For test.exp.Y, which is not a list
res_exp.Y <- process_test_results(test.exp.Y)

In [88]:
print(res_exp.X3.list)

[[1]]
           piX       h2X        iX
10 0.003166779 0.3384765 0.6738606

[[2]]
        piX       h2X        iX
5 0.0229665 0.5017473 0.7154573

[[3]]
         piX        h2X       iX
1 0.01778804 0.02470265 1.094882



In [89]:
# Function to extract the values from the results
extract_values <- function(res) {
  pi_X = as.numeric(res[1])
  h2_x = as.numeric(res[2])
  i_X = as.numeric(res[3])
  return(list(pi_X=pi_X, h2_x=h2_x, i_X=i_X))
}

# Extract values for each of the res_exp lists and dataframes
values_X1 <- lapply(res_exp.X1.list, extract_values)
values_X2 <- lapply(res_exp.X2.list, extract_values)
values_X3 <- lapply(res_exp.X3.list, extract_values)
values_Y <- extract_values(res_exp.Y)
#values_Y[[1]]=100/1000
#values_Y[[2]]=0.2

In [90]:
print(values_Y)
print(alp1)

$pi_X
[1] 0.2

$h2_x
[1] 0.01336014

$i_X
[1] 0.7695241

[1] 0.1


In [91]:
generate_starting_points <- function(values_X, values_Y, nSP, alp1, alp2, iXY) {
  h2_x = values_X$h2_x
  h2_y = values_Y$h2_x  # Use h2_x from values_Y
  
  # Generate the starting points
  sp_tX = runif(nSP, 0, 0.5)
  sp_tY = runif(nSP, -0.5, 0.5)
  sp_h2X = pmax(0, h2_x - (sp_tX^2))  # Changed to pmax to handle vector operations
  sp_h2Y = pmax(0, h2_y - (sp_tY^2))
  sp_axy = replicate(nSP, (alp1 + runif(1, -0.1, 0.1)))
  sp_ayx = replicate(nSP, (alp2 + runif(1, -0.1, 0.1)))
  sp_iXY = replicate(nSP, (iXY + runif(1, -0.05, 0.05)))
  
  return(list(sp_tX=sp_tX, sp_tY=sp_tY, sp_h2X=sp_h2X, sp_h2Y=sp_h2Y, sp_axy=sp_axy, sp_ayx=sp_ayx, sp_iXY=sp_iXY))
}

# Generate starting points for each of the values_X lists using values_Y
starting_points_X1 <- lapply(values_X1, generate_starting_points, values_Y=values_Y, nSP=nSP, alp1=alp1, alp2=alp2, iXY=iXY)
starting_points_X2 <- lapply(values_X2, generate_starting_points, values_Y=values_Y, nSP=nSP, alp1=alp1, alp2=alp2, iXY=iXY)
starting_points_X3 <- lapply(values_X3, generate_starting_points, values_Y=values_Y, nSP=nSP, alp1=alp1, alp2=alp2, iXY=iXY)


In [92]:
print(starting_points_X1)

[[1]]
[[1]]$sp_tX
 [1] 0.19853044 0.34182259 0.04966079 0.35057483 0.45427051 0.29586036
 [7] 0.01714191 0.22103247 0.11479056 0.08474881

[[1]]$sp_tY
 [1] -0.21296635  0.39834746  0.22464028  0.25698380 -0.09177540  0.39149702
 [7]  0.26432796 -0.17571021 -0.01263364 -0.26967139

[[1]]$sp_h2X
 [1] 0.2990622 0.2216338 0.3360103 0.2155738 0.1321148 0.2509432 0.3381827
 [8] 0.2896211 0.3252996 0.3312941

[[1]]$sp_h2Y
 [1] 0.000000000 0.000000000 0.000000000 0.000000000 0.004937416 0.000000000
 [7] 0.000000000 0.000000000 0.013200532 0.000000000

[[1]]$sp_axy
 [1] 0.035708815 0.081144225 0.079159678 0.027976734 0.153670859 0.136056748
 [7] 0.057351553 0.002518659 0.025077647 0.143851910

[[1]]$sp_ayx
 [1] 0.1646455 0.2216665 0.3057107 0.2357106 0.2145066 0.3020036 0.2360664
 [8] 0.3287734 0.2495831 0.2942107

[[1]]$sp_iXY
 [1] 1.0227763 1.0282612 1.0339950 1.0405493 1.0493847 1.0111305 1.0149527
 [8] 0.9683897 1.0116896 1.0183115


[[2]]
[[2]]$sp_tX
 [1] 0.13346375 0.01987861 0.31165752 0

In [93]:
# Define the function to construct the matrix from starting points
construct_matrix <- function(starting_point) {
  para <- cbind(starting_point$sp_h2X,
                starting_point$sp_h2Y,
                starting_point$sp_tX,
                starting_point$sp_tY,
                starting_point$sp_axy,
                starting_point$sp_ayx,
                starting_point$sp_iXY)
  
  sp_mat <- matrix(unlist(para), ncol=7, byrow=FALSE)
  colnames(sp_mat) <- c("sp_h2X", "sp_h2Y", "sp_tX", "sp_tY", "sp_axy", "sp_ayx", "sp_iXY")
  
  return(sp_mat)
}

# Apply the function to each list of starting points to get lists of matrices
matrices_X1 <- lapply(starting_points_X1, construct_matrix)
matrices_X2 <- lapply(starting_points_X2, construct_matrix)
matrices_X3 <- lapply(starting_points_X3, construct_matrix)

In [94]:
print(matrices_X1)

[[1]]
         sp_h2X      sp_h2Y      sp_tX       sp_tY      sp_axy    sp_ayx
 [1,] 0.2990622 0.000000000 0.19853044 -0.21296635 0.035708815 0.1646455
 [2,] 0.2216338 0.000000000 0.34182259  0.39834746 0.081144225 0.2216665
 [3,] 0.3360103 0.000000000 0.04966079  0.22464028 0.079159678 0.3057107
 [4,] 0.2155738 0.000000000 0.35057483  0.25698380 0.027976734 0.2357106
 [5,] 0.1321148 0.004937416 0.45427051 -0.09177540 0.153670859 0.2145066
 [6,] 0.2509432 0.000000000 0.29586036  0.39149702 0.136056748 0.3020036
 [7,] 0.3381827 0.000000000 0.01714191  0.26432796 0.057351553 0.2360664
 [8,] 0.2896211 0.000000000 0.22103247 -0.17571021 0.002518659 0.3287734
 [9,] 0.3252996 0.013200532 0.11479056 -0.01263364 0.025077647 0.2495831
[10,] 0.3312941 0.000000000 0.08474881 -0.26967139 0.143851910 0.2942107
         sp_iXY
 [1,] 1.0227763
 [2,] 1.0282612
 [3,] 1.0339950
 [4,] 1.0405493
 [5,] 1.0493847
 [6,] 1.0111305
 [7,] 1.0149527
 [8,] 0.9683897
 [9,] 1.0116896
[10,] 1.0183115

[[2]]
        

In [95]:
  #SP_list = list("iX"=i_X,"iY"=i_Y,"piX"=pi_X,"piY"=pi_Y,"sp_mat"=sp_mat)

  
  ## optimisation of said set of starting points
  bn = 2^7
  bins = 10
  param="comp" #possibility to develop nesting in later version
  w8s=wd #must be the same name as in likelihood function
  parscale2 = c(1e-1,1e-1,1e-1,1e-1,1e-1,1e-1,1e-1)
  nCores = 1 #for lapply: update to number wanted, or leave as NA to use third of available cores




In [96]:
pairTrait_twoStep_likelihood <- function(theta,betXY,w8s,m0,pi_U=0.1,pi_X,pi_Y,i_X,i_Y,nX,nY,zero_set="both",model="comp",bn=2^8,bins=15,M=1e7){

  piX = pi_X
  piU = pi_U #piU is unidentifiable
  piY = pi_Y
  h2X = abs(theta[1]);
  h2Y = abs(theta[2]);
  iX = i_X;
  iY = i_Y;

  if(model=="comp"){
    tX = abs(theta[3]);
    tY = theta[4];
    axy = theta[5];
    ayx = theta[6];
    iXY = theta[7];
  }

  if(model=="U"){
    tX = 0;
    tY = 0;
    axy = theta[3];
    ayx = theta[4];
    iXY = theta[5];
  }
    
  if(zero_set=="both"){
      axy=0
      ayx=0
      
  }
  if(zero_set=="axy"){
      axy=0
  }
  if(zero_set=="ayx"){
      ayx=0
  }


  sigX = sqrt(h2X/(piX*M))
  sigU = sqrt(1/(piU*M)); #h2U = 1
  sigY = sqrt(h2Y/(piY*M))

  m = nrow(betXY)

  Rp = matrix(c(iX/nX,iXY/sqrt(nX*nY),iXY/sqrt(nX*nY),iY/nY), nrow=2, ncol=2, byrow = T)
  bX = array(betXY[,1], c(1,1,m))
  bY = array(betXY[,2], c(1,1,m))

  minX = mean(bX)-(5*sd(bX));
  maxX = mean(bX)+(5*sd(bX));
  minY = mean(bY)-(5*sd(bY));
  maxY = mean(bY)+(5*sd(bY));

  dX = (maxX-minX)/(bn-1);
  dY = (maxY-minY)/(bn-1);

  minX = minX-dX/2;
  maxX = maxX+dX/2;
  minY = minY-dY/2;
  maxY = maxY+dY/2;

  bXi = ceiling((bX-minX)/dX);
  bYi = ceiling((bY-minY)/dY);

  bXi[bXi<1] = 1;
  bXi[bXi>bn] = bn;
  bYi[bYi<1] = 1;
  bYi[bYi>bn] = bn;

  if(max(c(piX,piU,piY)) > 0.2 || min(c(piX,piU,piY)) < 1e-6 || max(c((h2X+tX^2),(h2Y+tY^2)))>1
     || min(c((h2X+tX^2),(h2Y+tY^2))) < 1e-6|| abs(axy)>=1 || abs(ayx)>=1 || min(c(iX,iY))<=0.5
     || max(c(iX,iY))>=1.5 || abs(iXY)>1){
    logL = 1e10
    nmiss = 0
  }else{
    # min_pi1 = min(pi1)-1e-10;
    # max_pi1 = max(pi1)+1e-10;
    # dp = (max_pi1-min_pi1)/bins;
    # pc = min_pi1 + (dp * matrix(seq(0.5,(bins-0.5),1),ncol=1))
    # pix = ceiling((pi1-min_pi1)/dp);
    # min_sig1 = min(sig1)-1e-10;
    # max_sig1 = max(sig1)+1e-10;
    # ds = (max_sig1-min_sig1)/bins;
    # sc = min_sig1 + (ds * matrix(seq(0.5,(bins-0.5),1),ncol=1))
    # six = ceiling((sig1-min_sig1)/ds);
    # cix = pix + bins*(six-1);
    # uni_cix = sort(unique(cix))
    # ucix = match(uni_cix, cix)
    # ixMap = match(cix, uni_cix)
    # Sig1 = sc[six[ucix]];
    # Pi1 = pc[pix[ucix]];
    # mm = length(Sig1);

    # sigK = aperm(array(rep(Sig1, bn*mm), c(mm,bn,bn)), c(2,3,1))
    # piK = aperm(array(rep(Pi1, bn*mm), c(mm,bn,bn)), c(2,3,1))

    j = (0:(bn-1))
    vi = 2*pi*(j-bn/2)/(maxX-minX-dX);
    wj = 2*pi*(j-bn/2)/(maxY-minY-dY);
    # Rx = array(rep(vi, bn*mm), dim = c(bn, bn, mm))
    # Ry = aperm( array(rep(wj, bn*mm), dim = c(bn, bn, mm)),c(2,1,3))
    Rx = array(rep(vi, bn), dim = c(bn, bn))
    Ry = aperm( array(rep(wj, bn), dim = c(bn, bn)), c(2,1))

    # coX = (Rx+axy*Ry)*sigK;
    # coY = (Ry+ayx*Rx)*sigK;
    # coU = sigU * ((tX+ayx*tY)*Rx + (tY+axy*tX)*Ry) * sigK;
    coX = (Rx+axy*Ry)
    coY = (Ry+ayx*Rx)
    coU = sigU * ((tX+ayx*tY)*Rx + (tY+axy*tX)*Ry)

    # Lx = -m0*piX*( 1 - 1/sqrt(1+sigX^2*coX^2) )*piK;
    # Ly = -m0*piY*( 1 - 1/sqrt(1+sigY^2*coY^2) )*piK;
    # Lu = -m0*piU*( 1 - 1/sqrt(1+coU^2) )*piK;
    # e^{-\frac{1}{2}\sigma^2t^2}
    Lx = -m0*piX*( 1 - exp(-0.5*sigX^2*coX^2) );
    Ly = -m0*piY*( 1 - exp(-0.5*sigY^2*coY^2) );
    Lu = -m0*piU*( 1 - exp(-0.5*coU^2) );
    Le = -(1/2)*( (Rp[1,1]*Rx^2+Rp[2,2]*Ry^2) + 2*Rp[1,2]*(Rx*Ry) );
    # /!\ complex numbers here!
    mf_init = -2*log(as.complex(-1)) *
      ( matrix( rep( ((minX+dX/2)/(maxX-minX-dX))*j, length(j) ), nrow = length(j) , byrow=T) +
          ( matrix( rep( ((minY+dY/2)/(maxY-minY-dY))%*%t(j), length(j) ),  nrow = length(j) ) ) )
    # mf = array(rep(mf_init, mm), dim=c(bn, bn, mm))
    mf = array(rep(mf_init), dim=c(bn, bn))
    phi = exp(Lx+Ly+Lu+Le+mf);

    # FFT=array(NA, dim=c(bn, bn, mm))
    FFT=array(NA, dim=c(bn, bn))
    # for(l in 1:mm){
    #   FFT[,,l] = fft(phi[,,l])
    # }
    FFT = fft(phi)

    FFTmod_init = (1/((maxX-minX-dX)*(maxY-minY-dY)))*(as.complex(-1))^(bn*((minX+dX/2)/(maxX-minX-dX) +
                                                                              (minY+dY/2)/(maxY-minY-dY)) + (outer(j,j,FUN="+")) )

    # FFTmod = array(rep(FFTmod_init, mm), dim=c(bn,bn,mm));
    FFTmod = array(FFTmod_init, dim=c(bn,bn));
    FFT0 = Re(FFT*FFTmod);
    ixF = cbind(bXi, bYi);
    pfE = FFT0[ixF];
    selu = which(pfE>0);
    pfE = pfE[selu];
    my_w8s = w8s[selu]  #update weights vector if some SNPs are excluded
    nmiss = length(selu);

    lpfE = log(pfE);
    logL = -m * mean(log(pfE*my_w8s))
  }
  return(logL)
}

In [97]:
run_optimization_on_matrix <- function(matrix, values_X, values_Y, betX, betY, zero_set) {
  # Create a container to hold res_est values for each matrix
  all_res_est <- list()
  
  iX = values_X$i_X
  iY = values_Y$i_X
  piX = values_X$pi_X
  piY = values_Y$pi_X
    

  #for (i in seq_along(matrices_list)) {
    sp_mat <- matrix
    betXY <- cbind(betX, betY)
    # Generate a dataframe of lists for each row of parameters
    par.df = data.frame(par=I(apply(sp_mat, 1, as.list)))
    #iX=i_X; iY=i_Y; piX=pi_X; piY=pi_Y
    cat(print("Running optimisation"))
    #print(par.df[[1]])
    test.res <- parallel::mclapply(par.df[[1]], function(x) {
      theta = unlist(x)

      test = optim(theta, pairTrait_twoStep_likelihood,
                   betXY=betXY, w8s=w8s, M=M,
                   m0=m0, nX=nX, nY=nY, pi_U=0.1, pi_X=piX, pi_Y=piY, i_X=iX, i_Y=iY,
                   bn=2^7, bins=10,
                   method = "Nelder-Mead", zero_set=zero_set,
                   control = list(maxit = 5e3, parscale = parscale2))
      #print(test$value)
      list("mLL" = test$value, "par" = test$par, "conv" = test$convergence)
    }, mc.cores = nCores)

    test.res = as.data.frame(t(matrix(unlist(test.res), nrow = length(unlist(test.res[1])))))
    colnames(test.res) = c("mLL", "h2X", "h2Y", "tX", "tY", "axy", "ayx", "iXY", "conv")

    test.res %>%
      dplyr::mutate(h2X = abs(h2X),
                    h2Y = abs(h2Y),
                    tX = abs(tX)) -> res_values

    res_values = cbind("SP" = c(1:nrow(res_values)), "mLL" = res_values[, 1], "piX" = piX, "piY" = piY, res_values[, -1], "iX" = iX, "iY" = iY)
    #write.csv(res_values, paste0("FullResult_DataB", i, ".csv"), row.names = FALSE)
    
    res_est = res_values[which(res_values$mLL == min(res_values$mLL)),]
    res_est = dplyr::select(res_est, -c(SP, conv))
    #write.table(res_est, paste0("SummaryResult_DataB", i, ".csv"), sep = ",", row.names = FALSE, col.names = FALSE, append = TRUE)
    #print(res_values$mLL)
    #all_res_est[[i]] <- res_est
  #}
  
  return(res_est)
}


In [98]:
# Create empty lists to store the results for each X
results_X1_axy <- list()
results_X1_ayx <- list()
results_X1_both <- list()


  results_X1_axy[[1]] <- run_optimization_on_matrix(matrices_X1[[1]], values_X1[[1]], values_Y, betX1[[1]], betY, "axy")
  results_X1_ayx[[1]] <- run_optimization_on_matrix(matrices_X1[[1]], values_X1[[1]], values_Y, betX1[[1]], betY, "ayx")
  results_X1_both[[1]] <- run_optimization_on_matrix(matrices_X1[[1]], values_X1[[1]], values_Y, betX1[[1]], betY, "both")


[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation

In [101]:
print(results_X1_ayx)

[[1]]
      mLL         piX piY       h2X          h2Y           tX         tY
8 -905.34 0.003166779 0.2 0.6799143 4.008332e-05 0.0002864126 -0.4369589
        axy       ayx      iXY        iX        iY
8 0.1394447 0.5470851 0.990934 0.6738606 0.7695241



In [102]:
# Iterate and optimize for each X1 value
for (i in seq_along(values_X1)) {
    
  results_X1_axy[[i]] <- run_optimization_on_matrix(matrices_X1[[i]], values_X1[[i]], values_Y, betX1[[i]], betY, "axy")
  results_X1_ayx[[i]] <- run_optimization_on_matrix(matrices_X1[[i]], values_X1[[i]], values_Y, betX1[[i]], betY, "ayx")
  results_X1_both[[i]] <- run_optimization_on_matrix(matrices_X1[[i]], values_X1[[i]], values_Y, betX1[[i]], betY, "both")

}

[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation

In [103]:
results_X2_axy <- list()
results_X2_ayx <- list()
results_X2_both <- list()

# Iterate and optimize for each X2 value
for (i in seq_along(values_X2)) {
  #results_X2[[i]] <- run_optimization_on_matrix(matrices_X2[[i]], values_X2[[i]], values_Y, betX2[[i]], betY)
  results_X2_axy[[i]] <- run_optimization_on_matrix(matrices_X2[[i]], values_X2[[i]], values_Y, betX2[[i]], betY, "axy")
  results_X2_ayx[[i]] <- run_optimization_on_matrix(matrices_X2[[i]], values_X2[[i]], values_Y, betX2[[i]], betY, "ayx")
  results_X2_both[[i]] <- run_optimization_on_matrix(matrices_X2[[i]], values_X2[[i]], values_Y, betX2[[i]], betY, "both")

}


[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation

In [104]:

results_X3_axy <- list()
results_X3_ayx <- list()
results_X3_both <- list()


# Iterate and optimize for each X3 value
for (i in seq_along(values_X3)) {
  #results_X3[[i]] <- run_optimization_on_matrix(matrices_X3[[i]], values_X3[[i]], values_Y, betX3[[i]], betY)
  results_X3_axy[[i]] <- run_optimization_on_matrix(matrices_X3[[i]], values_X3[[i]], values_Y, betX3[[i]], betY, "axy")
  results_X3_ayx[[i]] <- run_optimization_on_matrix(matrices_X3[[i]], values_X3[[i]], values_Y, betX3[[i]], betY, "ayx")
  results_X3_both[[i]] <- run_optimization_on_matrix(matrices_X3[[i]], values_X3[[i]], values_Y, betX3[[i]], betY, "both")

}

[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation[1] "Running optimisation"
Running optimisation

In [105]:
print(results_X1_ayx)

[[1]]
      mLL         piX piY       h2X          h2Y           tX         tY
8 -905.34 0.003166779 0.2 0.6799143 4.008332e-05 0.0002864126 -0.4369589
        axy       ayx      iXY        iX        iY
8 0.1394447 0.5470851 0.990934 0.6738606 0.7695241

[[2]]
        mLL       piX piY       h2X          h2Y          tX          tY
7 -794.4868 0.0229665 0.2 0.3617124 9.578456e-07 0.006735243 0.007111497
        axy       ayx       iXY        iX        iY
7 0.1659398 0.4497916 0.9963393 0.7154573 0.7695241

[[3]]
         mLL        piX piY        h2X          h2Y        tX         tY
10 -626.2159 0.01778804 0.2 0.00025365 1.357001e-06 0.3098195 -0.3256046
         axy       ayx       iXY       iX        iY
10 0.0237655 0.2949795 0.9999047 1.094882 0.7695241



In [106]:
print(results_X1_ayx[[1]])

      mLL         piX piY       h2X          h2Y           tX         tY
8 -905.34 0.003166779 0.2 0.6799143 4.008332e-05 0.0002864126 -0.4369589
        axy       ayx      iXY        iX        iY
8 0.1394447 0.5470851 0.990934 0.6738606 0.7695241


In [107]:
print(results_X1_both[[1]])

        mLL         piX piY       h2X          h2Y           tX           tY
8 -915.2313 0.003166779 0.2 0.2905783 2.606652e-05 1.514785e-08 -0.003588919
        axy       ayx       iXY        iX        iY
8 0.7676914 -0.359751 0.9912836 0.6738606 0.7695241


In [108]:
print(matrices_X1[[1]])

         sp_h2X      sp_h2Y      sp_tX       sp_tY      sp_axy    sp_ayx
 [1,] 0.2990622 0.000000000 0.19853044 -0.21296635 0.035708815 0.1646455
 [2,] 0.2216338 0.000000000 0.34182259  0.39834746 0.081144225 0.2216665
 [3,] 0.3360103 0.000000000 0.04966079  0.22464028 0.079159678 0.3057107
 [4,] 0.2155738 0.000000000 0.35057483  0.25698380 0.027976734 0.2357106
 [5,] 0.1321148 0.004937416 0.45427051 -0.09177540 0.153670859 0.2145066
 [6,] 0.2509432 0.000000000 0.29586036  0.39149702 0.136056748 0.3020036
 [7,] 0.3381827 0.000000000 0.01714191  0.26432796 0.057351553 0.2360664
 [8,] 0.2896211 0.000000000 0.22103247 -0.17571021 0.002518659 0.3287734
 [9,] 0.3252996 0.013200532 0.11479056 -0.01263364 0.025077647 0.2495831
[10,] 0.3312941 0.000000000 0.08474881 -0.26967139 0.143851910 0.2942107
         sp_iXY
 [1,] 1.0227763
 [2,] 1.0282612
 [3,] 1.0339950
 [4,] 1.0405493
 [5,] 1.0493847
 [6,] 1.0111305
 [7,] 1.0149527
 [8,] 0.9683897
 [9,] 1.0116896
[10,] 1.0183115


In [109]:
print(results_X1_axy[[1]])

        mLL         piX piY       h2X          h2Y        tX        tY
8 -921.3014 0.003166779 0.2 0.1303678 0.0002871924 0.6512207 0.1193035
         axy       ayx      iXY        iX        iY
8 -0.3113552 0.7597627 0.999997 0.6738606 0.7695241


In [110]:
print(results_X2_axy)

[[1]]
        mLL         piX piY          h2X          h2Y           tX
9 -936.5027 0.003166779 0.2 9.994687e-07 8.271127e-07 2.314604e-05
             tY       axy       ayx       iXY        iX        iY
9 -0.0006287322 -1.786525 0.9088785 0.9995596 0.6738606 0.7695241

[[2]]
        mLL       piX piY       h2X         h2Y        tX         tY        axy
9 -787.9995 0.0229665 0.2 0.3028571 6.54952e-11 0.4420576 -0.2130328 0.06437168
        ayx       iXY        iX        iY
9 0.1916931 0.9987226 0.7154573 0.7695241

[[3]]
        mLL        piX piY       h2X        h2Y         tX         tY
9 -606.7061 0.01778804 0.2 0.1742333 0.01137602 0.08157163 -0.2074545
         axy       ayx iXY       iX        iY
9 0.06467445 0.3065308   1 1.094882 0.7695241



In [111]:
print(results_X2_ayx)

[[1]]
       mLL         piX piY       h2X         h2Y           tX         tY
9 -936.295 0.003166779 0.2 0.4276397 2.00481e-10 0.0008367335 0.00107528
         axy       ayx       iXY        iX        iY
9 0.02066295 0.4946892 0.9995244 0.6738606 0.7695241

[[2]]
        mLL       piX piY      h2X          h2Y        tX          tY       axy
8 -789.4338 0.0229665 0.2 0.250912 5.433776e-11 0.5048708 -0.02610938 0.2011733
        ayx       iXY        iX        iY
8 0.3330345 0.9998349 0.7154573 0.7695241

[[3]]
        mLL        piX piY        h2X          h2Y        tX         tY
9 -604.6395 0.01778804 0.2 0.01140087 3.410274e-06 0.1919086 -0.1974463
         axy       ayx       iXY       iX        iY
9 0.09270359 0.3451921 0.9733728 1.094882 0.7695241



In [112]:
print(results_X2_both)

[[1]]
        mLL         piX piY       h2X          h2Y           tX           tY
9 -936.2633 0.003166779 0.2 0.4006117 3.090638e-07 0.0006465676 0.0008313015
        axy       ayx       iXY        iX        iY
9 0.1979812 0.4358136 0.9995242 0.6738606 0.7695241

[[2]]
       mLL       piX piY       h2X          h2Y        tX        tY      axy
1 -788.944 0.0229665 0.2 0.3279452 4.884579e-11 0.4397919 0.2132816 0.136265
        ayx       iXY        iX        iY
1 0.2727785 0.9999987 0.7154573 0.7695241

[[3]]
       mLL        piX piY         h2X        h2Y        tX        tY       axy
4 -606.294 0.01778804 0.2 0.000213672 0.01264806 0.2087504 0.1709458 0.2043817
        ayx iXY       iX        iY
4 0.3450196   1 1.094882 0.7695241



In [113]:
print(results_X3_axy)

[[1]]
        mLL         piX piY      h2X          h2Y         tX         tY
5 -928.1712 0.003166779 0.2 0.440825 1.701475e-07 0.03652925 0.04999531
        axy      ayx       iXY        iX        iY
5 0.3427002 0.358535 0.9976943 0.6738606 0.7695241

[[2]]
        mLL       piX piY       h2X          h2Y       tX          tY       axy
6 -788.9346 0.0229665 0.2 0.3881547 7.289586e-11 0.406404 -0.01652683 0.1663555
        ayx       iXY        iX        iY
6 0.1894668 0.9997256 0.7154573 0.7695241

[[3]]
       mLL        piX piY        h2X          h2Y         tX        tY
2 -625.054 0.01778804 0.2 1.0875e-07 2.259643e-07 0.02042723 -0.016096
        axy       ayx      iXY       iX        iY
2 -0.579895 0.7670544 0.993726 1.094882 0.7695241



In [114]:
print(results_X3_ayx)

[[1]]
        mLL         piX piY       h2X          h2Y          tX          tY
5 -935.2916 0.003166779 0.2 0.3051601 1.247338e-08 0.001135371 0.001315513
        axy       ayx       iXY        iX        iY
5 0.2755289 0.5320183 0.9976961 0.6738606 0.7695241

[[2]]
        mLL       piX piY      h2X          h2Y         tX          tY
9 -781.2484 0.0229665 0.2 0.345123 5.365838e-08 0.03509583 -0.02577999
        axy       ayx       iXY        iX        iY
9 0.2476799 0.3944588 0.9894702 0.7154573 0.7695241

[[3]]
        mLL        piX piY          h2X          h2Y        tX         tY
2 -618.3345 0.01778804 0.2 5.012805e-07 1.585897e-07 0.1877316 -0.3628112
        axy       ayx       iXY       iX        iY
2 0.9999986 0.2457491 0.9937485 1.094882 0.7695241



In [115]:
print(results_X3_both)

[[1]]
        mLL         piX piY       h2X          h2Y         tX         tY
5 -930.6371 0.003166779 0.2 0.9984662 4.685808e-08 0.03916322 0.04101745
         axy       ayx      iXY        iX        iY
5 -0.3526561 0.3405001 0.997767 0.6738606 0.7695241

[[2]]
        mLL       piX piY      h2X          h2Y        tX        tY       axy
1 -788.9552 0.0229665 0.2 0.355852 4.454386e-11 0.3182623 0.5195039 0.1756001
        ayx       iXY        iX        iY
1 0.2537406 0.9999233 0.7154573 0.7695241

[[3]]
        mLL        piX piY          h2X          h2Y         tX         tY
2 -620.5815 0.01778804 0.2 4.807464e-07 9.658521e-07 0.01459275 -0.1187613
         axy       ayx       iXY       iX        iY
2 -0.1673914 0.5975541 0.9937424 1.094882 0.7695241

